In [ ]:
import os
import numpy as np
import nilearn
import glob
import matplotlib.pyplot as plt
import nibabel as nib
import pandas as pd
from nilearn.image import concat_imgs, index_img, smooth_img
from nilearn.image import resample_to_img
#from nilearn import plotting
from nilearn.input_data import NiftiMasker
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut, cross_val_score, permutation_test_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from nilearn import image
#from nilearn.plotting import plot_stat_map, show
from sklearn.dummy import DummyClassifier
from sklearn.cross_validation import LeaveOneLabelOut

# ---STEP 1---
#Concatenate the imagine data into a NIFTI-2 file.
#Note: the data does not fit into a NIFTI-1 file, due to large number of subs.

In [ ]:
#set basepath
basepath=os.path.join('/projects','niblab','data','eric_data','W1','imagine')
outpath = "/projects/niblab/nilearn_projects"
#make a list of the files to concat
all_func = glob.glob(os.path.join(basepath,'level1_grace_edit','cs*++.feat','filtered_func_data.nii.gz'))
half_func = all_func[:67]


# ---STEP 2---


In [ ]:
#load & prepare MRI data
#load, fxnl, anatomical, & mask for plotting
fmri_subjs=os.path.join(outpath, 'concatenated_imagine_67.nii')
average_ana=os.path.join(outpath,'CS_avg_mprage_image.nii.gz')
imag_mask=os.path.join(outpath,'power_roimask_4bi.nii.gz')

#plot mask (Power ROIs) over anatomical that is defined above
#plotting.plot_roi(imag_mask,bg_img=average_ana,cmap='Paired')

#load labels for the functional data
stim = os.path.join('/projects','niblab','scripts','nilean_stuff','label_67_sub.csv')
#labels = np.recfromcsv(stim, delimiter=",",encoding='UTF-8')
#print(labels)
#Its shape corresponds to the number of time-points times the number of voxels in the mask
func_df = pd.read_csv(stim, sep=",")
#Retrieve the behavioral targets, that we are going to predict in the decoding
#y_mask = labels['labels']
#subs = labels['subs']
y_mask =  func_df['labels']
subs = func_df['subs']


# ---STEP 3---
#feature selection

In [ ]:
#To keep only data corresponding to app food or unapp food, we create a mask of the samples belonging to the condition.
#condition_mask = np.logical_or(y_mask == b'app',y_mask == b'unapp')
condition_mask = func_df["labels"].isin(['app', 'unapp'])
print(condition_mask.shape)
#y = y_mask[condition_mask]
y = y_mask[condition_mask]
print(y.shape)

n_conditions = np.size(np.unique(y))
print(n_conditions)
#n_conditions = np.size(np.unique(y))
print(y.unique())
#session = func_df[condition_mask].to_records(index=False)
#print(session.dtype.name)
#prepare the fxnl data.
nifti_masker = NiftiMasker(mask_img=imag_mask,
                           smoothing_fwhm=4,standardize=True,
                           memory_level=0)

fmri_trans = nifti_masker.fit_transform(fmri_subjs)
print(fmri_trans)
X = fmri_trans[condition_mask]
subs = subs[condition_mask]

print(fmri_trans)
[[ 0.5087769   0.27887663  0.3514245  ...  2.459353    3.7311056
   3.9869728 ]
 [ 0.48385125  0.29370055  0.35507813 ...  2.5287416   3.6351027
   3.9535809 ]
 [ 0.5118701   0.34580636  0.34771594 ...  2.5323887   3.7883573
   4.0834546 ]
 ...
 [-0.20601459  1.0056752  -0.14867683 ... -0.24131215 -0.262366
  -0.26612085]
 [-0.22412705  1.0654575  -0.16013078 ... -0.24127465 -0.26207733
  -0.26604658]
 [-0.23273358  1.0431011  -0.17234245 ... -0.24135125 -0.26185337
  -0.26517907]]



# ---STEP 4---


In [ ]:
#setting prediction  & testing the classifer
#Define the prediction function to be used. Here we use a Support Vector Classification, with a linear kernel

svc = SVC(kernel='linear')
print(svc)
feature_selection = SelectKBest(f_classif, k=500)
# We have our classifier (SVC),
# our feature selection (SelectKBest),
# and now, we can plug them together in a *pipeline*
# that performs the two operations successively.
anova_svc = Pipeline([('anova',feature_selection), ('svc',svc)])
#fit the decoder and predict
anova_svc.fit(X, y)
y_pred = anova_svc.predict(X)
cv = LeaveOneLabelOut(subs[subs < 10 ])
k_range = [10, 15, 30, 50 , 150, 300, 500, 1000, 1500, 3000, 5000]
#cv_scores = cross_val_score(anova_svc, X[subs ==1], y[subs ==1])
cv_scores = []
scores_validation = []
cv_means =[]

